In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import traitlets
import ipywidgets as ipw
import aiidalab_widgets_base as awb
from widget_periodictable import PTableWidget

import sys
sys.path.append('/home/hungdt/works/IMMAD/codes/IMMAD')
from immad.apps.substitution import run_immad

class SetupStructureStep(ipw.VBox, awb.WizardAppWidgetStep):
    disabled = traitlets.Bool()
    configuration = traitlets.Dict()

    def __init__(self, **kwargs):
        # setup the widget for choosing the crystal structure
        # and atoms for substitution
        self.structure = awb.StructureManagerWidget(
            importers=[
                awb.StructureUploadWidget(title="From computer"),
                ],
            storable=False,
            node_class='StructureData'
            )
        
        # observe if user chooses atoms and update state
        self.structure.viewer.observe(self._update_state, ["selection"])
        
        # click on the "Confirm Structure" button locks the
        # current configuration and advance to the next step
        self.confirm_button = ipw.Button(description="Confirm Structure",
                                         disabled=True)
        self.confirm_button.on_click(self._confirm_structure)

        # We need to update the step's state
        # whenever the configuration is changed.
        self.observe(self._update_state, ["configuration"])

        super().__init__([self.structure, self.confirm_button], **kwargs)
        
    def _confirm_structure(self, button):
        "Confirm the pizza configuration and expose as trait."
        self.configuration = dict({
            'structure' : self.structure.structure,
            'substitutional' : self.structure.viewer.selection
            })
        
    def reset(self):
        with self.hold_trait_notifications():
            self.configuration = {}
            self.structure.viewer.selection = []
            
    @traitlets.default("state")
    def _default_state(self):
        return self.State.READY

    def _update_state(self, _=None):
        """Update the step's state based on traits and widget state.

        The step state influences the representation of the step 
        (e.g. the "icon") and whether the "Next step" button is enabled.
        """
        is_selected = len(self.structure.viewer.selection) != 0
        
        if self.configuration:
            self.state = self.State.SUCCESS
        elif is_selected:
            self.state = self.State.CONFIGURED
        else:
            self.state = self.State.READY

    @traitlets.observe("state")
    def _observe_state(self, change):
        # Enable the confirm button in case that some atoms
        # has been chosen for substitution
        with self.hold_trait_notifications():
            self.disabled = change["new"] == self.State.SUCCESS
            self.confirm_button.disabled = change["new"] is not self.State.CONFIGURED
        
    @traitlets.observe("disabled")
    def _observe_disabled(self, change):
        with self.hold_trait_notifications():
            for child in self.children:
                child.disabled = change["new"]

                
class ChooseSubstitutionStep(ipw.VBox, awb.WizardAppWidgetStep):
    disabled = traitlets.Bool()
    elements = traitlets.List()
    
    def __init__(self, **kwargs):
        self.periodic_widget = PTableWidget(states=1, 
                                            selected_colors=['green'], 
                                            disabled_elements=[],
                                            unselected_color='pink', 
                                            border_color='black', 
                                            width='20px')
        self.periodic_widget.observe(self._update_state, 'selected_elements')
        
        self.confirm_button = ipw.Button(description="Confirm Substitution",
                                         disabled=False)
        self.confirm_button.on_click(self._confirm_substitution)
        
        self.observe(self._update_state, ["elements"])
        
        super().__init__([self.periodic_widget, self.confirm_button], **kwargs)
        
    def reset(self):
        with self.hold_trait_notifications():
            self.elements = []
            self.periodic_widget.selected_elements = {}

    def _update_state(self, _=None):
        """Update the step's state based on the order status and
        configuration traits."""
        is_selected = len(self.periodic_widget.selected_elements) > 0
        if len(self.elements) > 0:
            self.state = self.State.SUCCESS
        elif is_selected:
            self.state = self.State.CONFIGURED
        else:
            self.state = self.State.READY
            
    def _confirm_substitution(self, button):
        "Confirm the pizza configuration and expose as trait."
        self.elements = self.periodic_widget.get_elements_by_state(0)
    
    @traitlets.default("state")
    def _default_state(self):
        return self.State.READY
        
    @traitlets.observe("state")
    def _observe_state(self, change):
        """Enable the order button once the step is
        in the "configured" state."""
        with self.hold_trait_notifications():
            self.disabled = change["new"] == self.State.SUCCESS
            self.confirm_button.disabled = change["new"] is not self.State.CONFIGURED
            
    @traitlets.observe("disabled")
    def _observe_disabled(self, change):
        with self.hold_trait_notifications():
            for child in self.children:
                child.disabled = change["new"]

                
class ReviewAndPredictStep(ipw.VBox, awb.WizardAppWidgetStep):
    # We use traitlets to connect the different steps.
    # Note that we can use dlinked transformations,
    # they do not need to be of the same type.
    configuration = traitlets.Dict()
    elements = traitlets.List()

    def __init__(self, **kwargs):
        # The pizza configuration is represented as a formatted dictionary.
        self.substituted_label = ipw.HTML('<h4>Substituted atoms</h4>')
        self.substituted_text = ipw.Output()
        
        self.substitution_label = ipw.HTML('<h4>Elements for substitutions</h4>')
        
        self.result_label = ipw.HTML('<h4>Results from the prediction</h4>')
        self.result_text = ipw.Output()

        # The second step has only function: executing the order
        # by clicking on this button.
        self.predict_button = ipw.Button(description="Predict", disabled=True)
        self.predict_button.on_click(self.predict)

        # We update the step's state whenever
        # there is a change to the configuration or the order status.
        self.observe(self._update_state, ["configuration", "elements"])

        super().__init__([self.substituted_label, self.substituted_text,
                          self.substitution_label, self.result_label, 
                          self.result_text, self.predict_button], **kwargs)
        
    def reset(self):
        with self.hold_trait_notifications():
            self.configuration = {}
            self.elements = []
            self.substitution_label.value = '<h4>Elements for substitutions</h4>'
            self.result_text.clear_output()

    @traitlets.observe("configuration")
    def _observe_configuration(self, change):
        "Format and show the pizza configuration."
        tmp = {
            'ID' : [],
            'Atom Name' : [],
            'Pos x' : [],
            'Pos y' : [],
            'Pos z' : [],
            'Mass' : [],
            'Atomic Charge' : []
        }
        if change["new"]:
            struct = change['new']['structure']
            subs = change['new']['substitutional']
            for ind in subs:
                atom = struct[ind]
                tmp['ID'].append(ind)
                tmp['Atom Name'].append(atom.symbol)
                tmp['Pos x'].append(atom.position[0])
                tmp['Pos y'].append(atom.position[1])
                tmp['Pos z'].append(atom.position[2])
                tmp['Mass'].append(atom.mass)
                tmp['Atomic Charge'].append(atom.charge)
        df = pd.DataFrame(tmp)
        df.set_index('ID', inplace=True)

        with self.substituted_text:
            self.substituted_text.clear_output()
            display(df)
        
    @traitlets.observe('elements')
    def _observe_elements(self, change):
        if change['new']:
            self.substitution_label.value += f'<pre> {", ".join(change["new"])} </pre>'

    def predict(self, button):
        "Submit the order and simulate the delivery."
        self.predict_button.disabled = True        
        self.results = run_immad(self.configuration['structure'], self.elements,
                                 self.configuration['substitutional'])
        struct = self.configuration['structure']
        with self.result_text:
            self.result_text.clear_output()
            if len(self.results) == 0:
                print('No optimal substitution found!')
            for i, sub in enumerate(self.results):
                n = sub["subs"][0]
                print(f'{i}: substitute {struct[n].symbol} ({n}) by '
                      f'{sub["subs"][1]}, score {sub["proba"]:.2f}')
        self.state = self.State.SUCCESS

    @traitlets.default("state")
    def _default_state(self):
        return self.State.READY
    
    def _update_state(self, _=None):
        """Update the step's state based on the order status and
        configuration traits."""
        if len(self.configuration) > 0 and len(self.elements) > 0:
            self.state = self.State.CONFIGURED
        else:
            self.state = self.State.INIT

    @traitlets.observe("state")
    def _observe_state(self, change):
        """Enable the order button once the step is
        in the "configured" state."""
        self.predict_button.disabled = change["new"] is not self.State.CONFIGURED


if __name__ == '__main__':
    pd.set_option("display.precision", 2)

    # Setup all steps of the app. 
    # Setting the `auto_advance` trait to True makes it such that 
    # the next step is automatically selected once the previous step is
    # in the "success" state.
    setup_structure_step = SetupStructureStep(auto_advance=True)
    choose_substitution_step = ChooseSubstitutionStep(auto_advance=True)
    review_and_predict_step = ReviewAndPredictStep(auto_advance=False)

    ipw.dlink(
        (setup_structure_step, "configuration"),
        (review_and_predict_step, "configuration"),
    )
    ipw.dlink(
        (choose_substitution_step, "elements"),
        (review_and_predict_step, "elements")
    )

    # Setup the app by adding the various steps in order.
    app = awb.WizardAppWidget(
        steps=[
            ("Setup crystal structure", setup_structure_step),
            ("Choose elements for substitution", choose_substitution_step),
            ("Review your choice and predict", review_and_predict_step)
        ]
    )
#    display(setup_structure_step)
    # Display the app to the user.
    display(app)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


WizardAppWidget(children=(HBox(children=(Button(description='Previous step', disabled=True, icon='step-backwar…